In [0]:
from google.colab import drive
drive.mount('/gdrive')
!git clone https://github.com/akg92/quora-question-duplicates.git


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
Cloning into 'quora-question-duplicates'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 81 (delta 24), reused 70 (delta 13), pack-reused 0
Unpacking objects: 100% (81/81), done.


In [0]:
!ls data
!pip install autocorrect

ls: cannot access 'data': No such file or directory
    100% |████████████████████████████████| 3.6MB 8.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/bf/b8/ae/704d5643f1d0637c5b87d9feccf2ee923c492b703bb0bfbb19
Successfully built autocorrect


In [0]:
import os
data_file = '/gdrive/My Drive/quora/train.csv'
lda_model = '/gdrive/My Drive/quora/lda/lda_model.h5'
os.chdir('./quora-question-duplicates')
!cd quora-question-duplicates

/bin/bash: line 0: cd: quora-question-duplicates: No such file or directory


In [0]:
from sklearn.model_selection import train_test_split
import pandas as pd
df = pd.read_csv(data_file)

trainDf,testDf = train_test_split(df, test_size=0.2,stratify=df['is_duplicate'].values)
trainDf.to_csv('data/train.csv',index=False)
testDf.to_csv('data/test.csv',index=False)



import os
import pandas as pd
def get_processed_df(data_dir='./data',file_suffix='processed.csv'):
    test_fp = os.path.join(data_dir,'test_'+file_suffix)
    train_fp = os.path.join(data_dir,'train_'+file_suffix)
    train_df = pd.read_csv(train_fp)
    # Drop NA values
    train_df = train_df.dropna()
    test_df = pd.read_csv(test_fp)
    test_df = test_df.dropna()
    return train_df,test_df
  


In [0]:
import sys
sys.path.append('./models')
from data_processing import clean_and_save
data_dir = './data'
test_file = os.path.join(data_dir,'test.csv')
train_file = os.path.join(data_dir,'train.csv')
test_processed_file = os.path.join(data_dir,'test_processed.csv')
train_processed_file  = os.path.join(data_dir,'train_processed.csv')
    #print(test_file)import sys
sys.path.append('./models')
    ## train and save the files
    ## to avoid spell coreect comment the below line
ENABLE_SPELL_CORRECT = False
""" for single process uncomment below"""
clean_and_save(test_file,test_processed_file)
clean_and_save(train_file,train_processed_file)
#!python models/data_processing.py
train_df,test_df = get_processed_df()

completed 10000
completed 20000
completed 30000
completed 40000
completed 50000
completed 60000
completed 70000
completed 80000
Finished Question1
completed 90000
completed 100000
completed 110000
completed 120000
completed 130000
completed 140000
completed 150000
completed 160000
Finished Question2
Finished processing file ./data/test.csv
completed 170000
completed 180000
completed 190000
completed 200000
completed 210000
completed 220000
completed 230000
completed 240000
completed 250000
completed 260000
completed 270000
completed 280000
completed 290000
completed 300000
completed 310000
completed 320000
completed 330000
completed 340000
completed 350000
completed 360000
completed 370000
completed 380000
completed 390000
completed 400000
completed 410000
completed 420000
completed 430000
completed 440000
completed 450000
completed 460000
completed 470000
completed 480000
Finished Question1
completed 490000
completed 500000
completed 510000
completed 520000
completed 530000
completed 

In [0]:
import sys
sys.path.append('./models')
from topic_modeling import build_topics_scores
train_df,test_df = build_topics_scores(train_df,test_df)

/usr/local/lib/python3.6/dist-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
train_Y = train_df['is_duplicate']
test_Y = test_df['is_duplicate']
test_df.drop(['is_duplicate'],axis=1,inplace=True)
train_df.drop(['is_duplicate'],axis=1,inplace=True)

In [0]:
from xgbmodel import XGBClassifier
from sklearn.model_selection import GridSearchCV
import pickle

params = {
        'min_child_weight': [1,3, 5],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }


params = {
        'min_child_weight': [1],
        'gamma': [0.5],
        'subsample': [0.6],
        'colsample_bytree': [0.6],
        'max_depth': [4]
        }
search = GridSearcCV(XGBClassifier(),params,cv=4)

search.fit(train_df,train_Y)

model_save_file = '/gdrive/My Drive/quora/lda/xgb.h5'
with open(model_save_file,'wb') as f:
  pickle.dump(search.best_estimator_,f)